In [112]:
#import statements
import pandas as pd
import numpy as np
from numpy import array
from numpy import asarray
from numpy import zeros
from pandas import DataFrame
import matplotlib.pyplot # visualization library
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

import re
import os
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
import sys
import warnings
import matplotlib.pyplot as plt
stop_words = set(stopwords.words('english'))


from tensorflow import keras
from keras.models import Model
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten
from keras.layers import GlobalMaxPooling1D
from keras.layers import Input
from keras.layers.embeddings import Embedding
from keras.layers.core import Dense, Dropout
from keras.layers.recurrent import LSTM
from keras.preprocessing.text import Tokenizer
from keras.layers.convolutional import Conv1D    
from keras.optimizers import Adam

import seaborn as sns

In [113]:
reviews = pd.read_csv('Restaurant_Reviews.csv')
reviews

,Review,Liked,category
0,Wow... Loved this place.,1,0
1,Crust is not good.,0,2
2,Not tasty and the texture was just nasty.,0,2
3,Stopped by during the late May bank holiday of...,1,0
4,The selection on the menu was great and so wer...,1,"2, 3"
...,...,...,...
990,I think food should have flavor and texture an...,0,2
991,Appetite instantly gone.,0,2
992,Overall I was not impressed and would not go b...,0,0
993,"The whole experience was underwhelming, and I ...",0,0


In [114]:
list(reviews.iloc[:,2])

['0',
 '2',
 '2',
 '0',
 '2, 3',
 '1',
 '2',
 '2',
 '2',
 '0',
 '1',
 '0',
 '0, 3',
 '2',
 '2, 4',
 '1',
 '0',
 '1',
 '0, 5',
 '0',
 '2',
 '2',
 '1',
 '1',
 '0',
 '2',
 '1, 2',
 '1, 2',
 '2',
 '3',
 '0',
 '0',
 '0',
 '0',
 '3',
 '2, 3',
 '1',
 '0',
 '1',
 '2',
 '0',
 '2',
 '1',
 '1, 2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '1',
 '2',
 '1',
 '2',
 '1, 2',
 '0, 5',
 '2',
 '2',
 '2',
 '1',
 '2',
 '2',
 '0',
 '0',
 '1, 2, 3',
 '0',
 '0',
 '1, 2',
 '0',
 '1, 2',
 '1',
 '1, 2',
 '0, 5',
 '2',
 '1',
 '2',
 '2',
 '2, 5',
 '0',
 '1',
 '2',
 '0',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '0',
 '0',
 '0',
 '2',
 '1',
 '0',
 '0',
 '1',
 '0',
 '2',
 '1',
 '1',
 '2, 3',
 '2',
 '0',
 '0',
 '1, 2, 3',
 '1',
 '0',
 '0',
 '2',
 '1',
 '0',
 '2',
 '3',
 '0',
 '4',
 '1',
 '2',
 '0, 1',
 '2',
 '0, 1',
 '1',
 '2',
 '2',
 '1',
 '0, 2',
 '1, 2',
 '0',
 '2, 5',
 '2, 3',
 '2',
 '2',
 '1',
 '2',
 '0',
 '2',
 '2, 5',
 '1, 5',
 '1',
 '0',
 '1',
 '1, 2',
 '2',
 '2',
 '1, 4',
 '1, 2',
 '1',
 '0',
 '1',
 '1',
 '0, 5',
 '1'

In [115]:
#create a list of lists for category column
category_list = list(reviews.iloc[:,2])
category_list = [list(x) for x in category_list]
for sub in category_list: 
    sub[:] = [ele for ele in sub if ele != ',']
for sub in category_list: 
    sub[:] = [ele for ele in sub if ele != ' ']
category_list = [[int(float(j)) for j in i] for i in category_list]
category_list

[[0],
 [2],
 [2],
 [0],
 [2, 3],
 [1],
 [2],
 [2],
 [2],
 [0],
 [1],
 [0],
 [0, 3],
 [2],
 [2, 4],
 [1],
 [0],
 [1],
 [0, 5],
 [0],
 [2],
 [2],
 [1],
 [1],
 [0],
 [2],
 [1, 2],
 [1, 2],
 [2],
 [3],
 [0],
 [0],
 [0],
 [0],
 [3],
 [2, 3],
 [1],
 [0],
 [1],
 [2],
 [0],
 [2],
 [1],
 [1, 2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [1],
 [2],
 [1],
 [2],
 [1, 2],
 [0, 5],
 [2],
 [2],
 [2],
 [1],
 [2],
 [2],
 [0],
 [0],
 [1, 2, 3],
 [0],
 [0],
 [1, 2],
 [0],
 [1, 2],
 [1],
 [1, 2],
 [0, 5],
 [2],
 [1],
 [2],
 [2],
 [2, 5],
 [0],
 [1],
 [2],
 [0],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [0],
 [0],
 [0],
 [2],
 [1],
 [0],
 [0],
 [1],
 [0],
 [2],
 [1],
 [1],
 [2, 3],
 [2],
 [0],
 [0],
 [1, 2, 3],
 [1],
 [0],
 [0],
 [2],
 [1],
 [0],
 [2],
 [3],
 [0],
 [4],
 [1],
 [2],
 [0, 1],
 [2],
 [0, 1],
 [1],
 [2],
 [2],
 [1],
 [0, 2],
 [1, 2],
 [0],
 [2, 5],
 [2, 3],
 [2],
 [2],
 [1],
 [2],
 [0],
 [2],
 [2, 5],
 [1, 5],
 [1],
 [0],
 [1],
 [1, 2],
 [2],
 [2],
 [1, 4],
 [1, 2],
 [1],
 [0],
 [1],
 [1],
 [0, 5],
 [1]

In [116]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(tokenizer=lambda x: x, lowercase=False)
sparse_matrix = cv.fit_transform(category_list)

# To transform to dense matrix
sparse_matrix = sparse_matrix.todense()

sparse_matrix_df = pd.DataFrame(sparse_matrix)
sparse_matrix_df

,0,1,2,3,4,5
0,1,0,0,0,0,0
1,0,0,1,0,0,0
2,0,0,1,0,0,0
3,1,0,0,0,0,0
4,0,0,1,1,0,0
...,...,...,...,...,...,...
990,0,0,1,0,0,0
991,0,0,1,0,0,0
992,1,0,0,0,0,0
993,1,0,0,0,0,0


In [117]:
reviews_wo_singlecol_category = reviews.iloc[:, 0:1]
reviews_wo_singlecol_category

,Review
0,Wow... Loved this place.
1,Crust is not good.
2,Not tasty and the texture was just nasty.
3,Stopped by during the late May bank holiday of...
4,The selection on the menu was great and so wer...
...,...
990,I think food should have flavor and texture an...
991,Appetite instantly gone.
992,Overall I was not impressed and would not go b...
993,"The whole experience was underwhelming, and I ..."


In [118]:
prepped_reviews = reviews_wo_singlecol_category.join(sparse_matrix_df)

In [119]:
def preprocess_text(sen):
    # Removing html tags
    sentence = remove_tags(sen)

    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)

    return sentence

TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    return TAG_RE.sub('', text)

X = []
sentences = list(reviews['Review'])
for sen in sentences:
    X.append(preprocess_text(sen))

In [19]:
categories = [0,1,2,3,4,5]
train, test = train_test_split(prepped_reviews, test_size=0.2, shuffle=True)
X_train = train.Review
X_test = test.Review

In [177]:
#LINEAR SVC
SVC_pipeline = Pipeline([
                ('tfidf', TfidfVectorizer(stop_words=stop_words)),
                ('clf', OneVsRestClassifier(LinearSVC(), n_jobs=1)),
            ])
for category in categories:
    print('... Processing {}'.format(category))
    # train the model using X_dtm & y
    SVC_pipeline.fit(X_train, train[category])
    # compute the testing accuracy
    prediction = SVC_pipeline.predict(X_test)
    print('Test accuracy is {}'.format(accuracy_score(test[category], prediction)))

... Processing 0
Test accuracy is 0.8341708542713567
... Processing 1
Test accuracy is 0.8994974874371859
... Processing 2
Test accuracy is 0.8341708542713567
... Processing 3
Test accuracy is 0.9748743718592965
... Processing 4
Test accuracy is 0.964824120603015
... Processing 5
Test accuracy is 0.9798994974874372


In [178]:
#LOGISTIC REGRESSION
LogReg_pipeline = Pipeline([
                ('tfidf', TfidfVectorizer(stop_words=stop_words)),
                ('clf', OneVsRestClassifier(LogisticRegression(solver='sag'), n_jobs=1)),
            ])
for category in categories:
    print('... Processing {}'.format(category))
    # train the model using X_dtm & y
    LogReg_pipeline.fit(X_train, train[category])
    # compute the testing accuracy
    prediction = LogReg_pipeline.predict(X_test)
    print('Test accuracy is {}'.format(accuracy_score(test[category], prediction)))

... Processing 0
Test accuracy is 0.8140703517587939
... Processing 1
Test accuracy is 0.8492462311557789
... Processing 2
Test accuracy is 0.7989949748743719
... Processing 3
Test accuracy is 0.9698492462311558
... Processing 4
Test accuracy is 0.9547738693467337
... Processing 5
Test accuracy is 0.9798994974874372


In [179]:
#NAIVE BAYES
NB_pipeline = Pipeline([
                ('tfidf', TfidfVectorizer(stop_words=stop_words)),
                ('clf', OneVsRestClassifier(MultinomialNB(
                    fit_prior=True, class_prior=None))),
            ])
for category in categories:
    print('... Processing {}'.format(category))
    # train the model using X_dtm & y
    NB_pipeline.fit(X_train, train[category])
    # compute the testing accuracy
    prediction = NB_pipeline.predict(X_test)
    print('Test accuracy is {}'.format(accuracy_score(test[category], prediction)))

... Processing 0
Test accuracy is 0.7788944723618091
... Processing 1
Test accuracy is 0.864321608040201
... Processing 2
Test accuracy is 0.7989949748743719
... Processing 3
Test accuracy is 0.9698492462311558
... Processing 4
Test accuracy is 0.9547738693467337
... Processing 5
Test accuracy is 0.9798994974874372


In [63]:
#multilabel text classification w multiple output layers: KERAS
X = []
sentences = list(reviews["Review"])
for sen in sentences:
    X.append(preprocess_text(sen))

y = prepped_reviews[[0, 1, 2, 3, 4, 5]]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)


In [64]:
# First output
y1_train = y_train[[0]].values
y1_test =  y_test[[0]].values

# Second output
y2_train = y_train[[1]].values
y2_test =  y_test[[1]].values

# Third output
y3_train = y_train[[2]].values
y3_test =  y_test[[2]].values

# Fourth output
y4_train = y_train[[3]].values
y4_test =  y_test[[3]].values

# Fifth output
y5_train = y_train[[4]].values
y5_test =  y_test[[4]].values

# Sixth output
y6_train = y_train[[5]].values
y6_test =  y_test[[5]].values

In [65]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

vocab_size = len(tokenizer.word_index) + 1

maxlen = 200

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

In [69]:
glove_file = open('glove.6B.50d.txt', encoding="utf8")

embeddings_dictionary = dict()

for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary[word] = vector_dimensions
glove_file.close()

embedding_matrix = zeros((vocab_size, 50))
for word, index in tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

In [74]:
input_1 = Input(shape=(maxlen,))
embedding_layer = Embedding(vocab_size, 50, weights=[embedding_matrix], trainable=False)(input_1)
LSTM_Layer1 = LSTM(128)(embedding_layer)

output1 = Dense(1, activation='sigmoid')(LSTM_Layer1)
output2 = Dense(1, activation='sigmoid')(LSTM_Layer1)
output3 = Dense(1, activation='sigmoid')(LSTM_Layer1)
output4 = Dense(1, activation='sigmoid')(LSTM_Layer1)
output5 = Dense(1, activation='sigmoid')(LSTM_Layer1)
output6 = Dense(1, activation='sigmoid')(LSTM_Layer1)

model = Model(inputs=input_1, outputs=[output1, output2, output3, output4, output5, output6])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

In [77]:
print(model.summary())

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 200)]        0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 200, 50)      88750       input_2[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LSTM)                   (None, 128)          91648       embedding_1[0][0]                
__________________________________________________________________________________________________
dense_6 (Dense)                 (None, 1)            129         lstm_1[0][0]                     
_______________________________________________________________________________________

In [81]:
history = model.fit(x=X_train, y=[y1_train, y2_train, y3_train, y4_train, y5_train, y6_train], batch_size=8192, epochs=5, verbose=1, validation_split=0.2)

Epoch 1/5
1/1 [==============================] - 1s 865ms/step - loss: 4.1589 - dense_6_loss: 0.6931 - dense_7_loss: 0.6931 - dense_8_loss: 0.6931 - dense_9_loss: 0.6931 - dense_10_loss: 0.6931 - dense_11_loss: 0.6931 - dense_6_acc: 0.7170 - dense_7_acc: 0.7311 - dense_8_acc: 0.5582 - dense_9_acc: 0.9434 - dense_10_acc: 0.9717 - dense_11_acc: 0.9796 - val_loss: 4.1452 - val_dense_6_loss: 0.6924 - val_dense_7_loss: 0.6921 - val_dense_8_loss: 0.6930 - val_dense_9_loss: 0.6898 - val_dense_10_loss: 0.6884 - val_dense_11_loss: 0.6894 - val_dense_6_acc: 0.6750 - val_dense_7_acc: 0.7437 - val_dense_8_acc: 0.5625 - val_dense_9_acc: 0.9438 - val_dense_10_acc: 0.9812 - val_dense_11_acc: 0.9563
Epoch 2/5
1/1 [==============================] - 0s 159ms/step - loss: 4.1449 - dense_6_loss: 0.6922 - dense_7_loss: 0.6922 - dense_8_loss: 0.6930 - dense_9_loss: 0.6898 - dense_10_loss: 0.6885 - dense_11_loss: 0.6893 - dense_6_acc: 0.7170 - dense_7_acc: 0.7311 - dense_8_acc: 0.5582 - dense_9_acc: 0.9434 -

In [82]:
score = model.evaluate(x=X_test, y=[y1_test, y2_test, y3_test, y4_test, y5_test, y6_test], verbose=1)

print("Test Score:", score[0])
print("Test Accuracy:", score[1])

7/7 [==============================] - 0s 30ms/step - loss: 4.0314 - dense_6_loss: 0.6855 - dense_7_loss: 0.6849 - dense_8_loss: 0.6917 - dense_9_loss: 0.6610 - dense_10_loss: 0.6497 - dense_11_loss: 0.6585 - dense_6_acc: 0.6884 - dense_7_acc: 0.7186 - dense_8_acc: 0.5729 - dense_9_acc: 0.9598 - dense_10_acc: 0.9749 - dense_11_acc: 0.9598
Test Score: 4.031351089477539
Test Accuracy: 0.6855047941207886


In [83]:
#multilabel classification w single output layer

In [120]:
X = []
sentences = list(prepped_reviews["Review"])
for sen in sentences:
    X.append(preprocess_text(sen))

category_labels = prepped_reviews[[0, 1, 2, 3, 4, 5]]
y = category_labels.values

In [121]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, shuffle = True)

In [122]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

maxlen = max([len(s.split()) for s in X_train])

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

vocab_size = len(tokenizer.word_index) + 1


X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

In [123]:
from numpy import array
from numpy import asarray
from numpy import zeros

embeddings_dictionary = dict()

glove_file = open('glove.6B.50d.txt', encoding="utf8")

for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary[word] = vector_dimensions
glove_file.close()

embedding_matrix = zeros((vocab_size, 50))
for word, index in tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

In [124]:
deep_inputs = Input(shape=(maxlen,))
embedding_layer = Embedding(vocab_size, 50, weights=[embedding_matrix], trainable=False)(deep_inputs)
LSTM_Layer_1 = LSTM(128)(embedding_layer)
dense_layer_1 = Dense(6, activation='sigmoid')(LSTM_Layer_1)
model = Model(inputs=deep_inputs, outputs=dense_layer_1)

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

In [125]:
print(model.summary())

Model: "functional_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 32)]              0         
_________________________________________________________________
embedding_5 (Embedding)      (None, 32, 50)            88500     
_________________________________________________________________
lstm_5 (LSTM)                (None, 128)               91648     
_________________________________________________________________
dense_15 (Dense)             (None, 6)                 774       
Total params: 180,922
Trainable params: 92,422
Non-trainable params: 88,500
_________________________________________________________________
None


In [126]:
history = model.fit(X_train, y_train, batch_size=128, epochs=10, verbose=2, validation_split=0.2)

Epoch 1/10
5/5 - 1s - loss: 0.6787 - acc: 0.3082 - val_loss: 0.6375 - val_acc: 0.3313
Epoch 2/10
5/5 - 0s - loss: 0.5556 - acc: 0.3899 - val_loss: 0.4679 - val_acc: 0.3125
Epoch 3/10
5/5 - 0s - loss: 0.4153 - acc: 0.3868 - val_loss: 0.4277 - val_acc: 0.3125
Epoch 4/10
5/5 - 0s - loss: 0.3881 - acc: 0.3868 - val_loss: 0.4161 - val_acc: 0.3125
Epoch 5/10
5/5 - 0s - loss: 0.3900 - acc: 0.3868 - val_loss: 0.4191 - val_acc: 0.3125
Epoch 6/10
5/5 - 0s - loss: 0.3879 - acc: 0.3868 - val_loss: 0.4265 - val_acc: 0.3125
Epoch 7/10
5/5 - 0s - loss: 0.3871 - acc: 0.3868 - val_loss: 0.4198 - val_acc: 0.3125
Epoch 8/10
5/5 - 0s - loss: 0.3854 - acc: 0.3868 - val_loss: 0.4151 - val_acc: 0.3125
Epoch 9/10
5/5 - 0s - loss: 0.3860 - acc: 0.3868 - val_loss: 0.4145 - val_acc: 0.3125
Epoch 10/10
5/5 - 0s - loss: 0.3859 - acc: 0.3884 - val_loss: 0.4165 - val_acc: 0.3125


In [127]:
score = model.evaluate(X_test, y_test, verbose=1)

print("Test Score:", score[0])
print("Test Accuracy:", score[1])

7/7 [==============================] - 0s 6ms/step - loss: 0.3777 - acc: 0.4623
Test Score: 0.37773197889328003
Test Accuracy: 0.4623115658760071
